# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
df= data[['CustomerID','ProductID','Quantity']].groupby(['CustomerID','ProductID']).sum()
df

Quantity
CustomerID ProductID          
33         23                1
           51                1
           53                1
           67                1
           70                1
...                        ...
98200      392              25
           408              25
           418              25
           422              25
           444              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
import numpy as np
cus_m=pd.pivot_table(df,values='Quantity',index=['ProductID'],columns=['CustomerID'],aggfunc=np.sum)
cus_m=cus_m.fillna(0)
cus_m

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,25.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0
450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,25.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [17]:
cus_d = pdist(cus_m.T, 'euclidean')
cus_d2=squareform(cus_d)
cus_d2

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [46]:
cus_sim = pd.DataFrame(1/(1 + cus_d2), index=cus_m.columns, columns=cus_m.columns)

top5=cus_sim[200].sort_values(ascending=False)[1:6]
display(top5)


CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [50]:
st1_sel=df.loc[list(top5.keys())]

st1_sel


Quantity
CustomerID ProductID          
1072       13                1
           22                1
           28                1
           29                1
           30                1
...                        ...
3909       417               1
           422               1
           432               1
           434               1
           451               1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [73]:
prod_sel=st1_sel.groupby(level=['ProductID']).sum()
prod_sel.sort_values(by=['Quantity'],ascending=False)

,Quantity
ProductID,
382,4
50,3
187,3
253,3
202,3
...,...
4,1
214,1
79,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [100]:
c200=df.loc[200]
res=pd.merge(c200,prod_sel,how='right',on=['ProductID'])
produc_rec=res.loc[res['Quantity_x'].isna()].sort_values(by=['Quantity_y'],ascending=False)[0:5]
list(produc_rec['Quantity_y'].keys())


[382, 202, 253, 50, 370]

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [101]:
recomm={}
customerid=list(cus_m.columns)
for cus in customerid:
    top5=cus_sim[cus].sort_values(ascending=False)[1:6]
    st1_sel=df.loc[list(top5.keys())]
    prod_sel=st1_sel.groupby(level=['ProductID']).sum()
    ccus=df.loc[cus]
    res=pd.merge(ccus,prod_sel,how='right',on=['ProductID'])
    produc_rec=res.loc[res['Quantity_x'].isna()].sort_values(by=['Quantity_y'],ascending=False)[0:5]
    recomm[cus]=list(produc_rec['Quantity_y'].keys())



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [111]:
df_recomm=pd.DataFrame(recomm)
df_recomm


,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,382,382,423,193,187,402,393,303,155,193,...,71,309,100,201,144,101,409,18,178,33
1,442,202,263,248,444,57,386,442,402,61,...,220,274,77,410,394,210,95,42,126,172
2,269,253,372,402,382,187,444,187,83,269,...,370,220,404,413,234,404,430,56,396,199
3,402,50,193,401,4,155,269,51,78,442,...,152,240,383,92,28,135,227,258,103,105
4,174,370,287,14,393,72,193,192,326,187,...,310,415,183,325,358,98,179,212,443,62


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [141]:
#funcion that receive the method and returns the dict of recommendation
def recommendation(method):
    cus2_d = pdist(cus_m.T, method)
    cus2_d2=squareform(cus2_d)
    cus2_sim = pd.DataFrame(1/(1 + cus2_d2), index=cus_m.columns, columns=cus_m.columns)
    recomm2={}
    customerid=list(cus_m.columns)
    for cus in customerid:
        top5=cus2_sim[cus].sort_values(ascending=False)[1:6]
        st1_sel=df.loc[list(top5.keys())]
        prod_sel=st1_sel.groupby(level=['ProductID']).sum()
        ccus=df.loc[cus]
        res=pd.merge(ccus,prod_sel,how='right',on=['ProductID'])
        produc_rec=res.loc[res['Quantity_x'].isna()].sort_values(by=['Quantity_y'],ascending=False)[0:5]
        recomm2[cus]=list(produc_rec['Quantity_y'].keys())
    return recomm2


In [144]:
# some methods to compare 
methods=['braycurtis','chebyshev','cityblock','cosine','kulsinski','mahalanobis']
diff_method={}
for method in methods:
    recomm2=recommendation(method)
    df_recomm2=pd.DataFrame(recomm2)
    # For sure this comparision could be improved... but it is a beginning
    compa=[(ele,df_recomm[ele]==df_recomm2[ele]) for ele in customerid]
    df_compa=pd.DataFrame(compa)
    df_compa.columns=['CustomerID','recommen']
    df_compa['result']=df_compa['recommen'].apply(sum)
    diff_method[method]=sum(df_compa['result']!=5)

# the diccionany gives the differences between methods with the euclidean.. 1000 is completly different 
diff_method

{'braycurtis': 1000,
 'chebyshev': 1000,
 'cityblock': 990,
 'cosine': 999,
 'kulsinski': 1000,
 'mahalanobis': 999}

In [146]:
recomm_chebyshev=recommendation('chebyshev')
df_cheb=pd.DataFrame(recomm_chebyshev)
display(df_cheb)
df_recomm


,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,44,291,23,23,23,382,8,23,382,44,...,120,102,136,277,6,113,101,102,154,82
1,234,162,44,306,44,281,151,44,342,246,...,158,113,277,280,332,136,113,113,276,411
2,197,340,162,14,14,439,88,187,44,162,...,303,136,102,308,331,102,277,277,67,276
3,87,193,14,162,306,348,401,142,351,14,...,169,277,113,136,349,277,39,39,323,184
4,101,90,306,44,232,424,179,87,350,91,...,9,247,39,278,351,39,263,247,368,35


,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,382,382,423,193,187,402,393,303,155,193,...,71,309,100,201,144,101,409,18,178,33
1,442,202,263,248,444,57,386,442,402,61,...,220,274,77,410,394,210,95,42,126,172
2,269,253,372,402,382,187,444,187,83,269,...,370,220,404,413,234,404,430,56,396,199
3,402,50,193,401,4,155,269,51,78,442,...,152,240,383,92,28,135,227,258,103,105
4,174,370,287,14,393,72,193,192,326,187,...,310,415,183,325,358,98,179,212,443,62
